In [11]:
!pip install TTS==0.22.0
!pip install transformers==4.40.0
!pip install moviepy yt-dlp pydub


ERROR: Ignored the following versions that require a different python version: 0.0.10.2 Requires-Python >=3.6.0, <3.9; 0.0.10.3 Requires-Python >=3.6.0, <3.9; 0.0.11 Requires-Python >=3.6.0, <3.9; 0.0.12 Requires-Python >=3.6.0, <3.9; 0.0.13.1 Requires-Python >=3.6.0, <3.9; 0.0.13.2 Requires-Python >=3.6.0, <3.9; 0.0.14.1 Requires-Python >=3.6.0, <3.9; 0.0.15 Requires-Python >=3.6.0, <3.9; 0.0.15.1 Requires-Python >=3.6.0, <3.9; 0.0.9 Requires-Python >=3.6.0, <3.9; 0.0.9.1 Requires-Python >=3.6.0, <3.9; 0.0.9.2 Requires-Python >=3.6.0, <3.9; 0.0.9a10 Requires-Python >=3.6.0, <3.9; 0.0.9a9 Requires-Python >=3.6.0, <3.9; 0.1.0 Requires-Python >=3.6.0, <3.10; 0.1.1 Requires-Python >=3.6.0, <3.10; 0.1.2 Requires-Python >=3.6.0, <3.10; 0.1.3 Requires-Python >=3.6.0, <3.10; 0.10.0 Requires-Python >=3.7.0, <3.11; 0.10.1 Requires-Python >=3.7.0, <3.11; 0.10.2 Requires-Python >=3.7.0, <3.11; 0.11.0 Requires-Python >=3.7.0, <3.11; 0.11.1 Requires-Python >=3.7.0, <3.11; 0.12.0 Requires-Python >=3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 85.1 MB/s eta 0:00:00


In [10]:
# Free Colab: AI Creepy Fact Video Generator (no paid APIs)
# Run in Google Colab. This script exposes a small Flask endpoint via ngrok so n8n can trigger it.

# 1) Install dependencies (first run only)
!pip install -qT tts==0.14.0 yt-dlp moviepy ffmpeg-python transformers[torch] accelerate sentencepiece flask-ngrok pydub
!apt-get update -qq && apt-get install -y -qq ffmpeg fonts-noto

# 2) Python code (copy into a single cell)
import os
import time
import json
import shutil
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import subprocess

from transformers import pipeline
from moviepy.editor import VideoFileClip, AudioFileClip, TextClip, CompositeVideoClip, concatenate_videoclips
from TTS.api import TTS

# ---------- CONFIG ----------
OUTPUT_DIR = '/content/auto_videos'
os.makedirs(OUTPUT_DIR, exist_ok=True)
NGROK_PORT = 5000
N8N_WEBHOOK_URL = os.environ.get('N8N_WEBHOOK_URL','')  # set if you want automatic upload to n8n

WIDTH, HEIGHT = 1080, 1920
FPS = 25
FACTS_COUNT = 5
VOICE_MODEL = 'tts_models/en/ljspeech/tacotron2-DDC'  # this gives a clear male-like voice on many installs; Coqui has many models
COQUI_MODEL = 'tts_models/en/vctk/vits'  # alternative (deeper voice) you can switch

# ---------- Text generation (open-source) ----------
# Use a lightweight text model to generate short facts. flan-t5-small/large can be used. Choose model depending on available RAM.
text_gen = pipeline('text2text-generation', model='google/flan-t5-large')

def generate_facts(topic, n=5):
    prompt = f"Write {n} short, creepy and factual one-line facts about {topic}. Each fact must be 6-16 words, punchy, and suitable for a short video. Return as JSON array."
    out = text_gen(prompt, max_length=256, do_sample=True, top_p=0.9, num_return_sequences=1)[0]['generated_text']
    # try to parse JSON; otherwise split heuristically
    try:
        parsed = json.loads(out)
        if isinstance(parsed, list):
            return parsed[:n]
    except Exception:
        # fallback: split by lines or sentences
        parts = [p.strip().strip('"') for p in out.replace('', '. ').split('.') if p.strip()]
        facts = []
        for p in parts:
            if len(facts) >= n:
                break
            if len(p.split()) >= 3:
                facts.append(p.strip())
        # final safety: pad with generic facts
        while len(facts) < n:
            facts.append(f"Creepy fact about {topic} {len(facts)+1}")
        return facts

# ---------- TTS via Coqui TTS ----------
# Initialize TTS (Coqui). This will download model weights the first time and can be slow but free.
print('Loading Coqui TTS model (may take a minute)...')
try:
    tts = TTS(COQUI_MODEL)
except Exception as e:
    # fallback to a different model if download fails
    tts = TTS(VOICE_MODEL)

def synthesize_voice(text, out_path):
    # Coqui TTS save
    tts.tts_to_file(text=text, file_path=out_path)
    return out_path

# ---------- Fetch free B-roll using yt-dlp (YouTube Creative Commons search) ----------
# We use yt-dlp's "ytsearch" to quickly find clips. Query should include "creativecommons" or filter on CC content manually.

def download_youtube_cc(query, max_count=3, max_seconds=10):
    # Use yt-dlp to download top results from YouTube search. We attempt to download mp4 and trim later.
    files = []
    for i in range(max_count):
        search = f"ytsearch{max_count}:{query} creative commons"
    # Use single command to download multiple results
    cmd = [
        'yt-dlp',
        f'ytsearch{max_count}:{query} creative commons',
        '--no-playlist',
        '--restrict-filenames',
        '--format', 'mp4',
        '--output', os.path.join(OUTPUT_DIR, 'ytclip_%(id)s.%(ext)s')
    ]
    try:
        subprocess.run(cmd, check=True)
    except Exception as e:
        print('yt-dlp failed:', e)
    # collect downloaded files
    for f in os.listdir(OUTPUT_DIR):
        if f.startswith('ytclip_') and f.endswith('.mp4'):
            files.append(os.path.join(OUTPUT_DIR, f))
    # trim or ensure durations
    final_files = []
    for f in files[:max_count]:
        try:
            clip = VideoFileClip(f)
            dur = min(10, clip.duration)
            outf = f.replace('.mp4', '_trim.mp4')
            clip.subclip(0, dur).write_videofile(outf, fps=FPS, codec='libx264', audio_codec='aac', threads=2, verbose=False, logger=None)
            final_files.append(outf)
            clip.close()
        except Exception:
            continue
    return final_files

# ---------- Compose vertical video ----------
from moviepy.video.fx.all import resize

def build_vertical_video(facts, topic, voice_path, broll_clips, out_path):
    # Create background: if multiple clips, concatenate/loop; otherwise create ColorClip fallback
    bg_clips = []
    for c in broll_clips:
        try:
            vc = VideoFileClip(c)
            vc = vc.resize(height=HEIGHT)
            # center-crop to WIDTH if necessary
            if vc.w < WIDTH:
                vc = vc.resize(width=WIDTH)
            vc = vc.subclip(0, min(6, vc.duration))
            bg_clips.append(vc)
        except Exception as e:
            continue
    if not bg_clips:
        # fallback: animated dark fog generated by a simple zoom on a dark image
        from moviepy.video.VideoClip import ColorClip
        bg = ColorClip((WIDTH, HEIGHT), color=(8,8,12)).set_duration(30)
    else:
        bg = concatenate_videoclips(bg_clips, method='compose').loop(duration=60)

    # Create text overlays for facts
    overlays = []
    start = 0.8
    per = 5
    for fact in facts:
        txt = TextClip(fact, fontsize=64, font='DejaVu-Sans', method='label', size=(WIDTH-160, None), align='center')
        txt = txt.set_position(('center', HEIGHT*0.15)).set_start(start).set_duration(per)
        overlays.append(txt)
        start += per

    # Add watermark username
    watermark = TextClip('darktruths.hub007', fontsize=28, font='DejaVu-Sans').set_pos(('right','bottom')).set_duration(sum([per for _ in facts])+2)

    audio = AudioFileClip(voice_path)
    duration = audio.duration

    final = CompositeVideoClip([bg] + overlays + [watermark], size=(WIDTH, HEIGHT)).set_duration(duration)
    final = final.set_audio(audio)
    final.write_videofile(out_path, fps=FPS, codec='libx264', audio_codec='aac', threads=4)
    # close clips
    try:
        for c in bg_clips:
            c.close()
    except Exception:
        pass
    return out_path

# ---------- Flask endpoint ----------
app = Flask(__name__)
run_with_ngrok(app)

@app.route('/generate', methods=['POST'])
def generate_endpoint():
    data = request.json or {}
    topic = data.get('topic') or 'creepy facts'
    # Generate facts
    facts = generate_facts(topic, n=FACTS_COUNT)
    # Synthesize voice (join facts with short pauses)
    utter = '. '.join(facts) + '.'
    voice_file = os.path.join(OUTPUT_DIR, f'voice_{int(time.time())}.wav')
    synthesize_voice(utter, voice_file)
    # Download B-roll (dark fog + abandoned places queries)
    broll = download_youtube_cc('dark fog cinematic', max_count=2)
    broll += download_youtube_cc('abandoned building cinematic', max_count=2)
    if not broll:
        print('No b-roll found — using fallback background')
    # Build video
    out_file = os.path.join(OUTPUT_DIR, f'creepy_{int(time.time())}.mp4')
    build_vertical_video(facts, topic, voice_file, broll, out_file)

    # Optionally upload to n8n webhook (if configured)
    if N8N_WEBHOOK_URL:
        import requests
        files = {'file': open(out_file, 'rb')}
        payload = {'topic': topic, 'username': 'darktruths.hub007'}
        try:
            r = requests.post(N8N_WEBHOOK_URL, files=files, data=payload, timeout=180)
            return jsonify({'status':'ok','n8n_status': r.status_code})
        except Exception as e:
            return jsonify({'status':'ok','local_path': out_file, 'error': str(e)})
    return jsonify({'status':'ok','local_path': out_file})

if __name__ == '__main__':
    app.run(port=NGROK_PORT)



Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -T
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


ModuleNotFoundError: No module named 'TTS'